In [1]:
#KTCL
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile")
driver.maximize_window()
time.sleep(5) 

# List
all_buses = []

# Iterate through pages
for page in range(1, 5):
    try:
        if page > 1:
            pagination_tab = WebDriverWait(driver, 100).until(
                EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
            )
            pagination_tab.click()
            time.sleep(5)
    except TimeoutException:
        print(f"Timeout while trying to locate pagination tab for page {page}")
        continue  # Move to the next page 
    

    # Scrape bus routes
    routes = driver.find_elements(By.CLASS_NAME, 'route')
    all_bus_routes_link = [route.get_attribute('href') for route in routes]
    all_bus_routes_name = [route.text.strip() for route in routes]

    # Iterate over each bus route link
    for link, name in zip(all_bus_routes_link, all_bus_routes_name):
        driver.get(link)
        time.sleep(5)  

        try:
            # Click the "View Buses" button
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            view_buses_button.click()
            time.sleep(5)  # Wait for buses to load

            # Scroling down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for more content to load

            # Collect bus details
            bus_name = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time= driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price = driver.find_elements(By.CLASS_NAME, "fare.d-block")
            seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

            # Store bus details in a list
            for i in range(len(bus_name)):
                bus_detail = {
                    "Route_Name": name,
                    "Route_Link": link,
                    "Bus_Name": bus_name[i].text,
                    "Bus_Type": bus_type[i].text,
                    "Departing_Time": departing_time[i].text,
                    "Duration": duration[i].text,
                    "Reaching_Time": reaching_time[i].text,
                    "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                    "Price": price[i].text,
                    "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'  
                }
                all_buses.append(bus_detail)

        except Exception as e:
            print(f"Error occurred while scraping bus details for {link}: {str(e)}")

# Close the driver
driver.quit()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_buses)

# Save the DataFrame to a CSV file
df.to_csv('kadamba_bus_info.csv', index=False)

Timeout while trying to locate pagination tab for page 2
Timeout while trying to locate pagination tab for page 3
Timeout while trying to locate pagination tab for page 4


In [2]:
df = pd.DataFrame(all_buses)
df.to_csv('kadamba_bus_info.csv', index=False)
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Atmaram Gobus,VE A/C Sleeper (2+1),21:00,10h 45m,07:45,4.7,INR 1199,21 Seats available
1,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Atmaram Gobus,VE A/C Sleeper (2+1),22:30,10h 15m,08:45,4.7,INR 1199,17 Seats available
2,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,IntrCity SmartBus,AC Sleeper (2+1),23:30,10h 20m,09:50,4.4,INR 2019,14 Seats available
3,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,AdZingbus Plus,VE A/C Sleeper (2+1),00:10,12h 55m,13:05,4.2,1291,26 Seats available
4,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,IntrCity SmartBus,Bharat Benz A/C Seater /Sleeper (2+1),21:00,11h 05m,08:05,4.4,INR 1019,15 Seats available
...,...,...,...,...,...,...,...,...,...,...
297,Solapur to Goa,https://www.redbus.in/bus-tickets/solapur-to-goa,New Shivbaba Tours And Travels,AC Sleeper (2+1),18:00,12h 30m,06:30,1.8,949,21 Seats available
298,Calangute (goa) to Goa Airport,https://www.redbus.in/bus-tickets/calangute-go...,Kadamba Transport Corporation Limited (KTCL) -...,AC Seater (2+2),10:00,01h 45m,11:45,4.0,INR 200,39 Seats available
299,Calangute (goa) to Goa Airport,https://www.redbus.in/bus-tickets/calangute-go...,Kadamba Transport Corporation Limited (KTCL) -...,Volvo AC Seater 2+2,13:30,01h 45m,15:15,3.5,INR 200,40 Seats available
300,Calangute (goa) to Goa Airport,https://www.redbus.in/bus-tickets/calangute-go...,Kadamba Transport Corporation Limited (KTCL) -...,AC Seater (2+2),15:30,01h 45m,17:15,4.1,INR 200,41 Seats available


In [1]:
#WB
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile")
driver.maximize_window()
time.sleep(5) 

# List
all_buses = []

# Iterate through pages
for page in range(1, 5):
    try:
        if page > 1:
            pagination_tab = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
            )
            pagination_tab.click()
            time.sleep(5)
    except TimeoutException:
        print(f"Timeout while trying to locate pagination tab for page {page}")
        continue  # Move to the next page 
    

    # Scrape bus routes
    routes = driver.find_elements(By.CLASS_NAME, 'route')
    all_bus_routes_link = [route.get_attribute('href') for route in routes]
    all_bus_routes_name = [route.text.strip() for route in routes]

    # Iterate over each bus route link
    for link, name in zip(all_bus_routes_link, all_bus_routes_name):
        driver.get(link)
        time.sleep(5)  

        try:
            # Click the "View Buses" button
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            view_buses_button.click()
            time.sleep(5)  # Wait for buses to load

            # Scroling down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for more content to load

            # Collect bus details
            bus_name = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time= driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price = driver.find_elements(By.CLASS_NAME, "fare.d-block")
            seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

            # Store bus details in a list
            for i in range(len(bus_name)):
                bus_detail = {
                    "Route_Name": name,
                    "Route_Link": link,
                    "Bus_Name": bus_name[i].text,
                    "Bus_Type": bus_type[i].text,
                    "Departing_Time": departing_time[i].text,
                    "Duration": duration[i].text,
                    "Reaching_Time": reaching_time[i].text,
                    "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                    "Price": price[i].text,
                    "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'  
                }
                all_buses.append(bus_detail)

        except Exception as e:
            print(f"Error occurred while scraping bus details for {link}: {str(e)}")

# Close the driver
driver.quit()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_buses)

# Save the DataFrame to a CSV file
df.to_csv('WB_bus_info.csv', index=False)

Timeout while trying to locate pagination tab for page 2
Timeout while trying to locate pagination tab for page 3
Timeout while trying to locate pagination tab for page 4


In [2]:
df = pd.DataFrame(all_buses)
df.to_csv('WB_bus_info.csv', index=False)
df


,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) HABRA-DIGHA via Esplanade - 176|04:00,Non AC Seater (2+3),04:00,05h 50m,09:50,3.3,INR 165,40 Seats available
1,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) BARASAT-DIGHA via Bally - 38|04:45,Non AC Seater (2+3),04:45,04h 50m,09:35,4.2,INR 161,41 Seats available
2,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) BARASAT-DIGHA via Bally - 38|05:40,Non AC Seater (2+3),05:40,04h 30m,10:10,3.2,INR 161,24 Seats available
3,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) BADURIA - DIGHA via Bally - 155|09:00,Non AC Seater (2+3),09:00,03h 25m,12:25,2.7,INR 161,39 Seats available
4,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) HABRA-DIGHA via Bally - 26|10:00,Non AC Seater (2+3),10:00,04h 45m,14:45,3.3,INR 161,37 Seats available
...,...,...,...,...,...,...,...,...,...,...
356,Barasat (West Bengal) to Contai (Kanthi),https://www.redbus.in/bus-tickets/barasat-west...,WBTC (CTC) HABRA-DIGHA via Bally - 26|14:00,Non AC Seater (2+3),14:00,03h 10m,17:10,3.6,INR 131,38 Seats available
357,Barasat (West Bengal) to Contai (Kanthi),https://www.redbus.in/bus-tickets/barasat-west...,WBTC (CTC) BARASAT-DIGHA via Bally - 38|15:00,Non AC Seater (2+3),15:00,02h 45m,17:45,3.3,INR 131,39 Seats available
358,Barasat (West Bengal) to Contai (Kanthi),https://www.redbus.in/bus-tickets/barasat-west...,City Express,AC Seater (2+3),05:35,03h 45m,09:20,3.3,INR 370,6 Seats available
359,Barasat (West Bengal) to Contai (Kanthi),https://www.redbus.in/bus-tickets/barasat-west...,Aradhana Bus Service,Non AC Seater (2+3),00:40,04h 20m,05:00,2.4,INR 350,24 Seats available


In [3]:
#NB
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/north-bengal-state-transport-corporation")
driver.maximize_window()
time.sleep(5) 

# List
all_buses = []

# Iterate through pages
for page in range(1, 5):
    try:
        if page > 1:
            pagination_tab = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
            )
            pagination_tab.click()
            time.sleep(5)
    except TimeoutException:
        print(f"Timeout while trying to locate pagination tab for page {page}")
        continue  # Move to the next page 



    # Scrape bus routes
    routes = driver.find_elements(By.CLASS_NAME, 'route')
    all_bus_routes_link = [route.get_attribute('href') for route in routes]
    all_bus_routes_name = [route.text.strip() for route in routes]

    # Iterate over each bus route link
    for link, name in zip(all_bus_routes_link, all_bus_routes_name):
        driver.get(link)
        time.sleep(5)  

        try:
            # Click the "View Buses" button
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            view_buses_button.click()
            time.sleep(5)  # Wait for buses to load

            # Scroling down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for more content to load

            # Collect bus details
            bus_name = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time= driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price = driver.find_elements(By.CLASS_NAME, "fare.d-block")
            seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

            # Store bus details in a list
            for i in range(len(bus_name)):
                bus_detail = {
                    "Route_Name": name,
                    "Route_Link": link,
                    "Bus_Name": bus_name[i].text,
                    "Bus_Type": bus_type[i].text,
                    "Departing_Time": departing_time[i].text,
                    "Duration": duration[i].text,
                    "Reaching_Time": reaching_time[i].text,
                    "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                    "Price": price[i].text,
                    "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'  
                }
                all_buses.append(bus_detail)

        except Exception as e:
            print(f"Error occurred while scraping bus details for {link}: {str(e)}")

# Close the driver
driver.quit()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_buses)

# Save the DataFrame to a CSV file
df.to_csv('NB_bus_info.csv', index=False)

Timeout while trying to locate pagination tab for page 2
Timeout while trying to locate pagination tab for page 3
Timeout while trying to locate pagination tab for page 4


In [4]:
df = pd.DataFrame(all_buses)
df.to_csv('NB_bus_info.csv', index=False)
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Siliguri to Darjeeling,https://www.redbus.in/bus-tickets/siliguri-to-...,NBSTC -SLG-ROH-DJL-02-8:00 - 158470,Non AC Seater 2+2,08:00,03h 00m,11:00,3.9,INR 105,3 Seats available
1,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...,NBSTC-COB-KOL-R1-13:30 - 156428,Non AC Seater 2+2,17:15,13h 15m,06:30,3.7,INR 486,14 Seats available
2,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...,NBSTC-SLG-RAI-KOL-RKT-18:00 - 176650,Non AC Seater (2+2),18:00,12h 00m,06:00,5.0,INR 486,19 Seats available
3,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...,NBSTC-SLG-KOL-R1-18:30 - 156410,NON A/C Seater Push Back (2+2),18:30,12h 00m,06:30,3.9,INR 486,10 Seats available
4,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...,NBSTC-SLG-KOL-AC-18:45 - 176648,AC Seater (2+2),18:45,12h 15m,07:00,3.6,INR 1245,30 Seats available
...,...,...,...,...,...,...,...,...,...,...
248,Kolkata to Balurghat,https://www.redbus.in/bus-tickets/kolkata-to-b...,Rubi Paribahan,Volvo 9600 A/C Seater/Sleeper (2+1),19:30,08h 30m,04:00,3.8,INR 750,36 Seats available
249,Kolkata to Balurghat,https://www.redbus.in/bus-tickets/kolkata-to-b...,New Dolphin Travels,NON A/C Seater / Sleeper (2+2),17:30,13h 30m,07:00,2.8,INR 450,23 Seats available
250,Kolkata to Balurghat,https://www.redbus.in/bus-tickets/kolkata-to-b...,Jio Dolphin Travels,NON A/C Seater / Sleeper (2+2),19:30,12h 30m,08:00,2.7,INR 400,27 Seats available
251,Kolkata to Balurghat,https://www.redbus.in/bus-tickets/kolkata-to-b...,Shree Krishna Travels,NON A/C Seater / Sleeper (2+2),18:45,12h 45m,07:30,2.5,INR 500,10 Seats available


In [5]:
#JK
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/jksrtc")
driver.maximize_window()
time.sleep(5) 

# List
all_buses = []

# Iterate through pages
for page in range(1, 5):
    try:
        if page > 1:
            pagination_tab = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
            )
            pagination_tab.click()
            time.sleep(5)
    except TimeoutException:
        print(f"Timeout while trying to locate pagination tab for page {page}")
        continue  # Move to the next page 

    # Scrape bus routes
    routes = driver.find_elements(By.CLASS_NAME, 'route')
    all_bus_routes_link = [route.get_attribute('href') for route in routes]
    all_bus_routes_name = [route.text.strip() for route in routes]

    # Iterate over each bus route link
    for link, name in zip(all_bus_routes_link, all_bus_routes_name):
        driver.get(link)
        time.sleep(5)  

        try:
            # Click the "View Buses" button
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            view_buses_button.click()
            time.sleep(5)  # Wait for buses to load

            # Scroling down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for more content to load

            # Collect bus details
            bus_name = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time= driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price = driver.find_elements(By.CLASS_NAME, "fare.d-block")
            seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

            # Store bus details in a list
            for i in range(len(bus_name)):
                bus_detail = {
                    "Route_Name": name,
                    "Route_Link": link,
                    "Bus_Name": bus_name[i].text,
                    "Bus_Type": bus_type[i].text,
                    "Departing_Time": departing_time[i].text,
                    "Duration": duration[i].text,
                    "Reaching_Time": reaching_time[i].text,
                    "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                    "Price": price[i].text,
                    "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'  
                }
                all_buses.append(bus_detail)

        except Exception as e:
            print(f"Error occurred while scraping bus details for {link}: {str(e)}")

# Close the driver
driver.quit()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_buses)

# Save the DataFrame to a CSV file
df.to_csv('JK_bus_info.csv', index=False)

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/jammu-to-poonch: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7CC05B095+29557]
	(No symbol) [0x00007FF7CBFCFA50]
	(No symbol) [0x00007FF7CBE8B56A]
	(No symbol) [0x00007FF7CBEDF695]
	(No symbol) [0x00007FF7CBEDF8EC]
	(No symbol) [0x00007FF7CBF2B777]
	(No symbol) [0x00007FF7CBF071CF]
	(No symbol) [0x00007FF7CBF2851C]
	(No symbol) [0x00007FF7CBF06F33]
	(No symbol) [0x00007FF7CBED116F]
	(No symbol) [0x00007FF7CBED22D1]
	GetHandleVerifier [0x00007FF7CC38C96D+3378253]
	GetHandleVerifier [0x00007FF7CC3D8497+3688311]
	GetHandleVerifier [0x00007FF7CC3CD1CB+3642539]
	GetHandleVerifier [0x00007FF7CC11A6B6+813462]
	(No symbol) [0x00007FF7CBFDAB5F]
	(No symbol) [0x00007FF7CBFD6B74]
	(No symbol) [0x00007FF7CBFD6D10]
	(No symbol) [0x00007FF7CBFC5C1F]
	BaseThreadInitThunk [0x00007FFAA478257D+29]
	RtlUserThreadStart [0x00007FFAA5CCAF28+40]

Error occurred while scraping bus details for https://www.redbus.in/bus-ticket

In [6]:
df = pd.DataFrame(all_buses)
df.to_csv('JK_bus_info.csv', index=False)
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Srinagar to Jammu (j and k),https://www.redbus.in/bus-tickets/srinagar-to-...,Jamna Travels-Jammu,Non AC Seater (2+2),19:00,11h 45m,06:45,2.3,600,30 Seats available
1,Srinagar to Jammu (j and k),https://www.redbus.in/bus-tickets/srinagar-to-...,New Pal Travels,NON A/C Seater / Sleeper (2+2),19:00,10h 00m,05:00,1.8,719,41 Seats available
2,Delhi to Srinagar,https://www.redbus.in/bus-tickets/delhi-to-sri...,Zingbus Plus,Bharat Benz A/C Seater /Sleeper (2+1),20:00,19h 20m,15:20,5.0,3584,28 Seats available
3,Delhi to Srinagar,https://www.redbus.in/bus-tickets/delhi-to-sri...,Kingfisher Busline Pvt Ltd,A/C Volvo B11R Multi-Axle Sleeper (2+1),16:00,17h 00m,09:00,5.0,INR 3399,36 Seats available
4,Delhi to Srinagar,https://www.redbus.in/bus-tickets/delhi-to-sri...,Zingbus Plus,A/C Sleeper (2+1),21:35,18h 55m,16:30,0,4057,16 Seats available
...,...,...,...,...,...,...,...,...,...,...
146,Kishtwar to Jammu (j and k),https://www.redbus.in/bus-tickets/kishtwar-to-...,Manpreet Travels,NON A/C Seater (2+2),04:00,08h 00m,12:00,2.4,INR 400,27 Seats available
147,Kishtwar to Jammu (j and k),https://www.redbus.in/bus-tickets/kishtwar-to-...,Manpreet Travels,NON A/C Seater (2+2),05:40,07h 50m,13:30,2.4,INR 400,27 Seats available
148,Kishtwar to Jammu (j and k),https://www.redbus.in/bus-tickets/kishtwar-to-...,Manpreet Travels,NON A/C Seater (2+2),07:00,08h 00m,15:00,2.4,INR 400,27 Seats available
149,Kishtwar to Jammu (j and k),https://www.redbus.in/bus-tickets/kishtwar-to-...,Manpreet Travels,NON A/C Seater (2+2),06:00,08h 30m,14:30,2.4,INR 400,27 Seats available


In [7]:
#KAAC
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/kaac-transport")
driver.maximize_window()
time.sleep(5) 

# List
all_buses = []

# Iterate through pages
for page in range(1, 5):
    try:
        if page > 1:
            pagination_tab = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
            )
            pagination_tab.click()
            time.sleep(5)
    except TimeoutException:
        print(f"Timeout while trying to locate pagination tab for page {page}")
        continue  # Move to the next page 
   

    # Scrape bus routes
    routes = driver.find_elements(By.CLASS_NAME, 'route')
    all_bus_routes_link = [route.get_attribute('href') for route in routes]
    all_bus_routes_name = [route.text.strip() for route in routes]

    # Iterate over each bus route link
    for link, name in zip(all_bus_routes_link, all_bus_routes_name):
        driver.get(link)
        time.sleep(5)  

        try:
            # Click the "View Buses" button
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            view_buses_button.click()
            time.sleep(5)  # Wait for buses to load

            # Scroling down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for more content to load

            # Collect bus details
            bus_name = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time= driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price = driver.find_elements(By.CLASS_NAME, "fare.d-block")
            seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

            # Store bus details in a list
            for i in range(len(bus_name)):
                bus_detail = {
                    "Route_Name": name,
                    "Route_Link": link,
                    "Bus_Name": bus_name[i].text,
                    "Bus_Type": bus_type[i].text,
                    "Departing_Time": departing_time[i].text,
                    "Duration": duration[i].text,
                    "Reaching_Time": reaching_time[i].text,
                    "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                    "Price": price[i].text,
                    "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'  
                }
                all_buses.append(bus_detail)

        except Exception as e:
            print(f"Error occurred while scraping bus details for {link}: {str(e)}")

# Close the driver
driver.quit()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_buses)

# Save the DataFrame to a CSV file
df.to_csv('KAAC_bus_info.csv', index=False)

Timeout while trying to locate pagination tab for page 2
Timeout while trying to locate pagination tab for page 3
Timeout while trying to locate pagination tab for page 4


In [8]:
df = pd.DataFrame(all_buses)
df.to_csv('KAAC_bus_info.csv', index=False)
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Diphu to Guwahati,https://www.redbus.in/bus-tickets/diphu-to-guw...,KAAC TRANSPORT - 157249,Non AC Seater 2+2,04:00,05h 30m,09:30,4.3,361,33 Seats available
1,Diphu to Guwahati,https://www.redbus.in/bus-tickets/diphu-to-guw...,Kareng Travels,Non AC Seater (2+2),04:30,05h 00m,09:30,4.0,361,26 Seats available
2,Diphu to Guwahati,https://www.redbus.in/bus-tickets/diphu-to-guw...,Kareng Travels,Bharat Benz A/C Seater (2+1),10:00,06h 00m,16:00,4.5,INR 400,18 Seats available
3,Guwahati to Diphu,https://www.redbus.in/bus-tickets/guwahati-to-...,Kareng Travels,Non AC Seater (2+2),11:45,06h 45m,18:30,4.5,361,6 Seats available
4,Dokmoka to Guwahati,https://www.redbus.in/bus-tickets/dokmoka-to-g...,KAAC TRANSPORT - 157249,Non AC Seater 2+2,05:45,03h 45m,09:30,4.3,INR 250,33 Seats available
5,Dokmoka to Guwahati,https://www.redbus.in/bus-tickets/dokmoka-to-g...,Kareng Travels,Non AC Seater (2+2),06:00,03h 30m,09:30,4.3,266,27 Seats available
6,Dokmoka to Guwahati,https://www.redbus.in/bus-tickets/dokmoka-to-g...,Kareng Travels,Bharat Benz A/C Seater (2+1),11:45,04h 15m,16:00,4.8,INR 300,18 Seats available
7,Dokmoka to Guwahati,https://www.redbus.in/bus-tickets/dokmoka-to-g...,Kareng Travels,Non AC Seater (2+1),13:00,03h 45m,16:45,4.0,INR 280,31 Seats available
8,Bokolia (assam) to Guwahati,https://www.redbus.in/bus-tickets/bokolia-assa...,KAAC TRANSPORT - 157249,Non AC Seater 2+2,05:00,04h 30m,09:30,3.8,INR 299,33 Seats available
9,Bokolia (assam) to Guwahati,https://www.redbus.in/bus-tickets/bokolia-assa...,Kareng Travels,Bharat Benz A/C Seater (2+1),11:15,04h 45m,16:00,4.2,INR 300,18 Seats available


In [9]:
#PUNJAB
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile")
driver.maximize_window()
time.sleep(5) 

# List
all_buses = []

# Iterate through pages
for page in range(1, 5):
    try:
        if page > 1:
            pagination_tab = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
            )
            pagination_tab.click()
            time.sleep(5)
    except TimeoutException:
        print(f"Timeout while trying to locate pagination tab for page {page}")
        continue  # Move to the next page 
   

    # Scrape bus routes
    routes = driver.find_elements(By.CLASS_NAME, 'route')
    all_bus_routes_link = [route.get_attribute('href') for route in routes]
    all_bus_routes_name = [route.text.strip() for route in routes]

    # Iterate over each bus route link
    for link, name in zip(all_bus_routes_link, all_bus_routes_name):
        driver.get(link)
        time.sleep(5)  

        try:
            # Click the "View Buses" button
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            view_buses_button.click()
            time.sleep(5)  # Wait for buses to load

            # Scroling down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for more content to load

            # Collect bus details
            bus_name = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time= driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price = driver.find_elements(By.CLASS_NAME, "fare.d-block")
            seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

            # Store bus details in a list
            for i in range(len(bus_name)):
                bus_detail = {
                    "Route_Name": name,
                    "Route_Link": link,
                    "Bus_Name": bus_name[i].text,
                    "Bus_Type": bus_type[i].text,
                    "Departing_Time": departing_time[i].text,
                    "Duration": duration[i].text,
                    "Reaching_Time": reaching_time[i].text,
                    "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                    "Price": price[i].text,
                    "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'  
                }
                all_buses.append(bus_detail)

        except Exception as e:
            print(f"Error occurred while scraping bus details for {link}: {str(e)}")

# Close the driver
driver.quit()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_buses)

# Save the DataFrame to a CSV file
df.to_csv('PUNJAB_bus_info.csv', index=False)

Timeout while trying to locate pagination tab for page 2
Timeout while trying to locate pagination tab for page 3
Timeout while trying to locate pagination tab for page 4


In [10]:
df = pd.DataFrame(all_buses)
df.to_csv('PUNJAB_bus_info.csv', index=False)
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...,PEPSU (Punjab) - 67085,Volvo AC Seater (2+2),02:00,06h 00m,08:00,4.0,INR 826,31 Seats available
1,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...,PEPSU (Punjab) - 67085,Volvo AC Seater (2+2),03:00,05h 00m,08:00,4.5,INR 711,28 Seats available
2,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...,PEPSU (Punjab) - 67070,Volvo AC Seater (2+2),06:16,05h 29m,11:45,3.6,INR 711,37 Seats available
3,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...,PEPSU (Punjab) - 67069,Volvo AC Seater (2+2),06:40,06h 50m,13:30,4.9,INR 826,27 Seats available
4,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...,PEPSU (Punjab) - 67069,Volvo AC Seater (2+2),08:00,05h 30m,13:30,4.3,INR 711,23 Seats available
...,...,...,...,...,...,...,...,...,...,...
437,Jalandhar to Delhi Airport,https://www.redbus.in/bus-tickets/jalandhar-to...,Armaan Tour & Travels,A/C Seater / Sleeper (2+1),21:30,08h 30m,06:00,1.9,INR 699,48 Seats available
438,Jalandhar to Delhi Airport,https://www.redbus.in/bus-tickets/jalandhar-to...,Travel Point,A/C Seater / Sleeper (2+2),22:25,06h 55m,05:20,2.7,INR 999,50 Seats available
439,Jalandhar to Delhi Airport,https://www.redbus.in/bus-tickets/jalandhar-to...,Golden Temple Volvo Bus Service,A/C Seater / Sleeper (2+2),00:25,07h 30m,07:55,1.7,INR 686,22 Seats available
440,Jalandhar to Delhi Airport,https://www.redbus.in/bus-tickets/jalandhar-to...,Indo Canadian Transport Pvt. Ltd.,Mercedes A/C Semi Sleeper (2+2),10:00,09h 00m,19:00,5.0,INR 3300,37 Seats available


In [11]:
#CTU
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu")
driver.maximize_window()
time.sleep(5) 

# List
all_buses = []

# Iterate through pages
for page in range(1, 5):
    try:
        if page > 1:
            pagination_tab = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
            )
            pagination_tab.click()
            time.sleep(5)
    except TimeoutException:
        print(f"Timeout while trying to locate pagination tab for page {page}")
        continue  # Move to the next page 

    # Scrape bus routes
    routes = driver.find_elements(By.CLASS_NAME, 'route')
    all_bus_routes_link = [route.get_attribute('href') for route in routes]
    all_bus_routes_name = [route.text.strip() for route in routes]

    # Iterate over each bus route link
    for link, name in zip(all_bus_routes_link, all_bus_routes_name):
        driver.get(link)
        time.sleep(5)  

        try:
            # Click the "View Buses" button
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            view_buses_button.click()
            time.sleep(5)  # Wait for buses to load

            # Scroling down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for more content to load

            # Collect bus details
            bus_name = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time= driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price = driver.find_elements(By.CLASS_NAME, "fare.d-block")
            seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

            # Store bus details in a list
            for i in range(len(bus_name)):
                bus_detail = {
                    "Route_Name": name,
                    "Route_Link": link,
                    "Bus_Name": bus_name[i].text,
                    "Bus_Type": bus_type[i].text,
                    "Departing_Time": departing_time[i].text,
                    "Duration": duration[i].text,
                    "Reaching_Time": reaching_time[i].text,
                    "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                    "Price": price[i].text,
                    "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'  
                }
                all_buses.append(bus_detail)

        except Exception as e:
            print(f"Error occurred while scraping bus details for {link}: {str(e)}")

# Close the driver
driver.quit()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_buses)

# Save the DataFrame to a CSV file
df.to_csv('CTU_bus_info.csv', index=False)

Timeout while trying to locate pagination tab for page 2
Timeout while trying to locate pagination tab for page 3
Timeout while trying to locate pagination tab for page 4


In [12]:
df = pd.DataFrame(all_buses)
df.to_csv('CTU_bus_info.csv', index=False)
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...,Chandigarh Transport Undertaking (CTU) - 165754,HVAC Seater (2+3),06:20,02h 45m,09:05,4.2,INR 195.6,43 Seats available
1,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...,Chandigarh Transport Undertaking (CTU) - 165750,HVAC Seater (2+3),12:20,02h 45m,15:05,3.9,INR 195.6,47 Seats available
2,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...,Chandigarh Transport Undertaking (CTU) - 165752,HVAC Seater (2+3),14:20,02h 45m,17:05,4.0,INR 195.6,47 Seats available
3,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...,Zimindara Travels,AC Sleeper (2+1),20:30,04h 00m,00:30,4.1,INR 799,32 Seats available
4,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...,Shree Mahaveer Travels Jaipur,VE A/C Seater / Sleeper (2+1),02:10,03h 20m,05:30,3.6,INR 499,36 Seats available
...,...,...,...,...,...,...,...,...,...,...
385,Chandigarh to Pathankot,https://www.redbus.in/bus-tickets/chandigarh-t...,KN Nehra Travels,A/C Seater/Sleeper (2+1),01:15,04h 45m,06:00,0,INR 599,16 Seats available
386,Chandigarh to Pathankot,https://www.redbus.in/bus-tickets/chandigarh-t...,KN Nehra Travels,A/C Seater/Sleeper (2+1),01:30,04h 30m,06:00,0,INR 599,16 Seats available
387,Chandigarh to Pathankot,https://www.redbus.in/bus-tickets/chandigarh-t...,Goel Tourist (Regd),A/C Seater / Sleeper (2+2),02:30,04h 20m,06:50,0,INR 788,44 Seats available
388,Chandigarh to Pathankot,https://www.redbus.in/bus-tickets/chandigarh-t...,Shub Yatra Holidays,A/C Sleeper (2+2),04:00,02h 55m,06:55,0,INR 849,5 Seats available


In [13]:
#ASTC
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/astc/?utm_source=rtchometile")
driver.maximize_window()
time.sleep(5) 

# List
all_buses = []

# Iterate through pages
for page in range(1, 5):
    try:
        if page > 1:
            pagination_tab = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
            )
            pagination_tab.click()
            time.sleep(5)
    except TimeoutException:
        print(f"Timeout while trying to locate pagination tab for page {page}")
        continue  # Move to the next page 
   

    # Scrape bus routes
    routes = driver.find_elements(By.CLASS_NAME, 'route')
    all_bus_routes_link = [route.get_attribute('href') for route in routes]
    all_bus_routes_name = [route.text.strip() for route in routes]

    # Iterate over each bus route link
    for link, name in zip(all_bus_routes_link, all_bus_routes_name):
        driver.get(link)
        time.sleep(5)  

        try:
            # Click the "View Buses" button
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            view_buses_button.click()
            time.sleep(5)  # Wait for buses to load

            # Scroling down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for more content to load

            # Collect bus details
            bus_name = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time= driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price = driver.find_elements(By.CLASS_NAME, "fare.d-block")
            seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

            # Store bus details in a list
            for i in range(len(bus_name)):
                bus_detail = {
                    "Route_Name": name,
                    "Route_Link": link,
                    "Bus_Name": bus_name[i].text,
                    "Bus_Type": bus_type[i].text,
                    "Departing_Time": departing_time[i].text,
                    "Duration": duration[i].text,
                    "Reaching_Time": reaching_time[i].text,
                    "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                    "Price": price[i].text,
                    "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'  
                }
                all_buses.append(bus_detail)

        except Exception as e:
            print(f"Error occurred while scraping bus details for {link}: {str(e)}")

# Close the driver
driver.quit()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_buses)

# Save the DataFrame to a CSV file
df.to_csv('ASTC_bus_info.csv', index=False)

Timeout while trying to locate pagination tab for page 2
Timeout while trying to locate pagination tab for page 3
Timeout while trying to locate pagination tab for page 4


In [14]:
df = pd.DataFrame(all_buses)
df.to_csv('ASTC_bus_info.csv', index=False)
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 147480,Bharat Benz A/C Seater (2+2),05:00,04h 00m,09:00,3.8,INR 270,8 Seats available
1,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 135927,Volvo AC Seater 2+2,07:10,04h 30m,11:40,4.3,INR 298,25 Seats available
2,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 156885,Bharat Benz A/C Seater (2+2),07:35,04h 25m,12:00,2.9,INR 298,33 Seats available
3,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 157959,Bharat Benz A/C Seater (2+2),14:00,04h 30m,18:30,4.1,INR 298,24 Seats available
4,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 154930,Volvo AC Seater 2+2,15:45,04h 15m,20:00,3.9,INR 298,25 Seats available
...,...,...,...,...,...,...,...,...,...,...
102,Jorhat to Dibrugarh,https://www.redbus.in/bus-tickets/jorhat-to-di...,SHIVAM TRAVELS,Bharat Benz A/C Seater (2+1),13:30,03h 30m,17:00,3.9,INR 580,19 Seats available
103,Jorhat to Dibrugarh,https://www.redbus.in/bus-tickets/jorhat-to-di...,Christina Travels,NON A/C Seater (2+2),12:30,04h 00m,16:30,2.7,420,35 Seats available
104,Jorhat to Dibrugarh,https://www.redbus.in/bus-tickets/jorhat-to-di...,Bashudev Transline (Under ASTC),A/C Seater (2+1),14:10,03h 50m,18:00,4.3,INR 550,14 Seats available
105,North Lakhimpur to Jorhat,https://www.redbus.in/bus-tickets/north-lakhim...,Padmanav Travels,NON A/C Seater Push Back (2+2),05:30,08h 05m,13:35,3.2,561,39 Seats available


In [15]:
#HRTC
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile")
driver.maximize_window()
time.sleep(5) 

# List
all_buses = []

# Iterate through pages
for page in range(1, 5):
    try:
        if page > 1:
            pagination_tab = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
            )
            pagination_tab.click()
            time.sleep(5)
    except TimeoutException:
        print(f"Timeout while trying to locate pagination tab for page {page}")
        continue  # Move to the next page 
    

    # Scrape bus routes
    routes = driver.find_elements(By.CLASS_NAME, 'route')
    all_bus_routes_link = [route.get_attribute('href') for route in routes]
    all_bus_routes_name = [route.text.strip() for route in routes]

    # Iterate over each bus route link
    for link, name in zip(all_bus_routes_link, all_bus_routes_name):
        driver.get(link)
        time.sleep(5)  

        try:
            # Click the "View Buses" button
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            view_buses_button.click()
            time.sleep(5)  # Wait for buses to load

            # Scroling down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for more content to load

            # Collect bus details
            bus_name = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time= driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price = driver.find_elements(By.CLASS_NAME, "fare.d-block")
            seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

            # Store bus details in a list
            for i in range(len(bus_name)):
                bus_detail = {
                    "Route_Name": name,
                    "Route_Link": link,
                    "Bus_Name": bus_name[i].text,
                    "Bus_Type": bus_type[i].text,
                    "Departing_Time": departing_time[i].text,
                    "Duration": duration[i].text,
                    "Reaching_Time": reaching_time[i].text,
                    "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                    "Price": price[i].text,
                    "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'  
                }
                all_buses.append(bus_detail)

        except Exception as e:
            print(f"Error occurred while scraping bus details for {link}: {str(e)}")

# Close the driver
driver.quit()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_buses)

# Save the DataFrame to a CSV file
df.to_csv('HRTC_bus_info.csv', index=False)

Timeout while trying to locate pagination tab for page 2
Timeout while trying to locate pagination tab for page 3
Timeout while trying to locate pagination tab for page 4


In [16]:
df = pd.DataFrame(all_buses)
df.to_csv('HRTC_bus_info.csv', index=False)
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 69,Himsuta AC Seater Volvo/Scania 2+2,00:40,08h 50m,09:30,4.7,INR 912,21 Seats available
1,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 6,Himsuta AC Seater Volvo/Scania 2+2,06:45,09h 25m,16:10,4.5,INR 912,29 Seats available
2,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 592,A/C Executive (2+3),08:05,10h 05m,18:10,3.5,INR 632,41 Seats available
3,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 129,Ordinary 3+2 Non AC Seater,08:50,09h 50m,18:40,2.6,INR 512,35 Seats available
4,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 7,Himsuta AC Seater Volvo/Scania 2+2,09:25,09h 05m,18:30,4.4,INR 912,20 Seats available
...,...,...,...,...,...,...,...,...,...,...
252,Chandigarh to Dharamshala (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...,Laxmi holidays,Volvo 9600 A/C Semi Sleeper (2+2),23:55,06h 04m,05:59,4.5,1567,15 Seats available
253,Delhi to Chamba (Himachal Pradesh),https://www.redbus.in/bus-tickets/delhi-to-cha...,HRTC - 531,Ordinary 3+2 Non AC Seater,17:50,16h 10m,10:00,3.5,INR 876,27 Seats available
254,Delhi to Chamba (Himachal Pradesh),https://www.redbus.in/bus-tickets/delhi-to-cha...,HRTC - 590,A/C Executive (2+3),19:00,15h 15m,10:15,3.1,INR 1062,33 Seats available
255,Delhi to Chamba (Himachal Pradesh),https://www.redbus.in/bus-tickets/delhi-to-cha...,HRTC - 84,Himsuta AC Seater Volvo/Scania 2+2,20:15,15h 15m,11:30,3.9,INR 1704,8 Seats available


In [17]:
#SBSTC
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile")
driver.maximize_window()
time.sleep(5) 

# List
all_buses = []

# Iterate through pages
for page in range(1, 5):
    try:
        if page > 1:
            pagination_tab = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
            )
            pagination_tab.click()
            time.sleep(5)
    except TimeoutException:
        print(f"Timeout while trying to locate pagination tab for page {page}")
        continue  # Move to the next page 
    

    # Scrape bus routes
    routes = driver.find_elements(By.CLASS_NAME, 'route')
    all_bus_routes_link = [route.get_attribute('href') for route in routes]
    all_bus_routes_name = [route.text.strip() for route in routes]

    # Iterate over each bus route link
    for link, name in zip(all_bus_routes_link, all_bus_routes_name):
        driver.get(link)
        time.sleep(5)  

        try:
            # Click the "View Buses" button
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            view_buses_button.click()
            time.sleep(5)  # Wait for buses to load

            # Scroling down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for more content to load

            # Collect bus details
            bus_name = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time= driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price = driver.find_elements(By.CLASS_NAME, "fare.d-block")
            seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

            # Store bus details in a list
            for i in range(len(bus_name)):
                bus_detail = {
                    "Route_Name": name,
                    "Route_Link": link,
                    "Bus_Name": bus_name[i].text,
                    "Bus_Type": bus_type[i].text,
                    "Departing_Time": departing_time[i].text,
                    "Duration": duration[i].text,
                    "Reaching_Time": reaching_time[i].text,
                    "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                    "Price": price[i].text,
                    "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'  
                }
                all_buses.append(bus_detail)

        except Exception as e:
            print(f"Error occurred while scraping bus details for {link}: {str(e)}")

# Close the driver
driver.quit()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_buses)

# Save the DataFrame to a CSV file
df.to_csv('SBSTC_bus_info.csv', index=False)

Timeout while trying to locate pagination tab for page 2
Timeout while trying to locate pagination tab for page 3
Timeout while trying to locate pagination tab for page 4


In [18]:
df = pd.DataFrame(all_buses)
df.to_csv('SBSTC_bus_info.csv', index=False)
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-DURGAPUR - KOLKATA - 05:00 (DGP DEPOT) -...,Non AC Seater (2+3),05:00,03h 30m,08:30,3.9,INR 150,50 Seats available
1,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-DURGAPUR - KARUNAMOYEE - 05:15 (DGP DEPO...,Non AC Seater (2+3),05:15,04h 00m,09:15,4.1,INR 155,43 Seats available
2,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-DURGAPUR - KOLKATA - 05:30 (DGP DEPOT) -...,Non AC Seater (2+3),05:30,03h 30m,09:00,3.6,INR 150,51 Seats available
3,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-DURGAPUR - KOLKATA - L/S - 06:00 (DGP DE...,Non AC Seater (2+3),06:00,03h 30m,09:30,3.0,INR 150,50 Seats available
4,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...,SBSTC-DURGAPUR - KARUNAMOYEE - 06:15 (DGP DEPO...,Non AC Seater (2+3),06:15,04h 00m,10:15,3.6,INR 155,44 Seats available
...,...,...,...,...,...,...,...,...,...,...
478,Kolkata to Bankura,https://www.redbus.in/bus-tickets/kolkata-to-b...,SBSTC-KARUNAMOYEE - BANKURA - VIA - DURGAPUR -...,Non AC Seater (2+3),12:00,05h 30m,17:30,2.3,INR 185,44 Seats available
479,Kolkata to Bankura,https://www.redbus.in/bus-tickets/kolkata-to-b...,SBSTC-KOLKATA - PURULIA - VIA - BANKURA - 13:0...,Non AC Seater (2+3),13:00,04h 00m,17:00,2.6,INR 180,41 Seats available
480,Kolkata to Bankura,https://www.redbus.in/bus-tickets/kolkata-to-b...,SBSTC-KOLKATA - BANKURA - 14:30 (Bankura Depot...,Non AC Seater (2+3),14:30,04h 00m,18:30,2.2,INR 180,51 Seats available
481,Kolkata to Bankura,https://www.redbus.in/bus-tickets/kolkata-to-b...,SBSTC-KOLKATA - BANKURA - 15:30 (Bankura Depot...,Non AC Seater (2+3),15:30,04h 00m,19:30,2.5,INR 180,52 Seats available
